In [1]:
from google.colab import drive 
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab\ Notebooks

!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Colab Notebooks


In [2]:
import os
import math
import numpy as np
import cv2 as cv
import keras
import tensorflow as tf

import tensorflow_datasets as tfds
from tensorflow.keras import applications
import tensorflow.keras.applications.vgg16 as vgg16
import tensorflow.keras.applications.vgg19 as vgg19

from keras import optimizers
from keras.preprocessing import image
from keras.utils import to_categorical

datasets, info = tfds.load(name="imagenet2012_subset", 
                        with_info=True, 
                        as_supervised=True,
                        data_dir="/content/drive/MyDrive/Colab Notebooks/datasets")

C = info.features['label'].num_classes
Ntrain = info.splits['train'].num_examples
Nvalidation = info.splits['validation'].num_examples
Nbatch = 64

train_dataset, validation_dataset = datasets['train'], datasets['validation']

def imagenet_generator(dataset, batch_size=Nbatch, num_classes=1000, is_training=False):
  images = np.zeros((batch_size, 224, 224, 3))
  labels = np.zeros((batch_size, 1))
  while True:
    count = 0 
    for sample in tfds.as_numpy(dataset):
      image = sample[0]
      label = sample[1]
    
      images[count%batch_size] = vgg16.preprocess_input(np.expand_dims(cv.resize(image, (224, 224)), 0))
    #   labels[count%batch_size] = np.expand_dims(to_categorical(label, num_classes=num_classes), 0)
      labels[count%batch_size] = label

      count += 1
      if (count%batch_size == 0):
        yield images, labels

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=True)
model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])

score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                  steps= Nvalidation // Nbatch, 
                                  verbose=1)
print(score)

553467904/553467096 [==============================] - 4s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 311s 344ms/step - loss: 8187333.8728 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.0, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]


In [ ]:
model = vgg19.VGG19(weights='imagenet', include_top=True)
model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])

score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                  steps= Nvalidation // Nbatch, 
                                  verbose=1)
print(score)

574717952/574710816 [==============================] - 5s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 333s 372ms/step - loss: 8182904.5806 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]


In [ ]:
import os

print(os.getcwd())
# print(len(os.listdir("./models")))
skip = ["vgg16_0.01_0.7_16_500_500", "vgg19_0.01_0.7_16_500_500", "vgg16_0.01_0.8_16_500_500", "vgg19_0.01_0.8_16_500_500", "vgg16_0.01_0.9_16_500_500", "vgg19_0.01_0.9_16_500_500",
        "vgg16_0.001_0.7_16_500_500", "vgg19_0.001_0.7_16_500_500", "vgg16_0.001_0.8_16_500_500", "vgg19_0.001_0.8_16_500_500", ]
for name in os.listdir("./models"):
  if name not in skip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  del model

/content/drive/My Drive/Colab Notebooks
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.01, sim threshold: 0.7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 219s 280ms/step - loss: 19185051.4258 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.2912e-04 - sparse_top_k_categorical_accuracy: 0.0048
[19289242.0, 0.0, 0.0009403008734807372, 0.004781530238687992]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg19, weight >= 16MB, block size: 500 x 500, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 252s 322ms/step - loss: 7014271.6579 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.0352e-04 - sparse_top_k_categorical_accuracy: 0.0051
[7048375.5, 0.0, 0.0009803137509152293, 0.0053617157973349094]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 219s 280ms/step - loss: 6284870.3613 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.3185e-04 - sparse_top_k_categorical_accuracy: 0.0045
[6312123.5, 0.0, 0.0008602752932347357, 0.004821543116122484]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg19, weight >= 16MB, block size: 500 x 500, fp threshold: 0.01, sim threshold: 0.8


...

781/781 [==============================] - 245s 313ms/step - loss: 3642950.8798 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 7.6322e-04 - sparse_top_k_categorical_accuracy: 0.0040
[3659974.25, 0.0, 0.0010403329506516457, 0.0048015364445745945]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 217s 277ms/step - loss: 6340329.9175 - accuracy: 5.1623e-04 - sparse_categorical_accuracy: 0.5607 - sparse_top_k_categorical_accuracy: 0.7980
[6363060.5, 0.0005801856750622392, 0.5590589046478271, 0.7969550490379333]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg19, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 246s 314ms/step - loss: 5818045.4220 - accuracy: 6.1071e-04 - sparse_categorical_accuracy: 0.4870 - sparse_top_k_categorical_accuracy: 0.7329
[5843653.5, 0.0007602432742714882, 0.4861955940723419, 0.7324743866920471]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 217s 277ms/step - loss: 5365490.3971 - accuracy: 6.9964e-04 - sparse_categorical_accuracy: 0.5533 - sparse_top_k_categorical_accuracy: 0.7916
[5385442.5, 0.000720230455044657, 0.5523567795753479, 0.790372908115387]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg19, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 252s 321ms/step - loss: 5223478.2820 - accuracy: 8.1220e-04 - sparse_categorical_accuracy: 0.4791 - sparse_top_k_categorical_accuracy: 0.7281
[5248467.5, 0.0007802496547810733, 0.47771286964416504, 0.7251720428466797]
-----------------------------------------------------------------------------------------------------------------------
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.9


In [ ]:
import os

print(os.getcwd())
# print(len(os.listdir("./models")))
skip = ["vgg16_0.01_0.7_16_500_500", "vgg19_0.01_0.7_16_500_500", "vgg16_0.01_0.8_16_500_500", "vgg19_0.01_0.8_16_500_500", "vgg16_0.01_0.9_16_500_500", "vgg19_0.01_0.9_16_500_500",
        "vgg16_0.001_0.7_16_500_500", "vgg19_0.001_0.7_16_500_500", "vgg16_0.001_0.8_16_500_500", "vgg19_0.001_0.8_16_500_500", ]
nskip = ["vgg16_0.001_0.9_16_500_500", "vgg19_0.001_0.9_16_500_500", "vgg16_0.01_0.7_16_600_600", "vgg19_0.01_0.7_16_600_600", "vgg16_0.01_0.8_16_600_600", "vgg19_0.01_0.8_16_600_600",
         "vgg16_0.01_0.9_16_600_600", "vgg19_0.01_0.9_16_600_600", "vgg16_0.001_0.7_16_600_600",
         "vgg19_0.001_0.7_16_600_600", "vgg16_0.001_0.8_16_600_600"]
for name in os.listdir("./models"):
  if name in skip or name not in nskip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  del model

/content/drive/My Drive/Colab Notebooks
vgg16_0.001_0.9_16_500_500
Model: vgg16, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.9


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 262s 282ms/step - loss: 8124500.8817 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6573 - sparse_top_k_categorical_accuracy: 0.8664
[8149460.5, 0.0009202944929711521, 0.6550096273422241, 0.8650768399238586]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_500_500
Model: vgg19, weight >= 16MB, block size: 500 x 500, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 251s 321ms/step - loss: 8088988.4335 - accuracy: 0.0010 - sparse_categorical_accuracy: 0.6639 - sparse_top_k_categorical_accuracy: 0.8686
[8113265.5, 0.0010003200732171535, 0.6611315608024597, 0.8677176833152771]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.7_16_600_600
Model: vgg16, weight >= 16MB, block size: 600 x 600, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 218s 279ms/step - loss: 22844358.8696 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.8340e-04 - sparse_top_k_categorical_accuracy: 0.0046
[22962818.0, 0.0, 0.0010003200732171535, 0.005001600366085768]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.01, sim threshold: 0.7


...

781/781 [==============================] - 219s 280ms/step - loss: 17082421.0716 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.7951e-04 - sparse_top_k_categorical_accuracy: 0.0046
[17185474.0, 0.0, 0.0010003200732171535, 0.004901568405330181]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.9_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.01, sim threshold: 0.9


781/781 [==============================] - 251s 321ms/step - loss: 5648955.6234 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.4305e-04 - sparse_top_k_categorical_accuracy: 0.0051
[5680225.5, 0.0, 0.0009603073121979833, 0.00504161324352026]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.7_16_600_600
Model: vgg16, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 219s 280ms/step - loss: 8187333.8830 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.7_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 253s 322ms/step - loss: 8182904.5710 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_600_600
Model: vgg16, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 222s 283ms/step - loss: 8187333.8983 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.8_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.8


In [ ]:
import os

print(os.getcwd())
# print(len(os.listdir("./models")))
skip = ["vgg16_0.01_0.7_16_500_500", "vgg19_0.01_0.7_16_500_500", "vgg16_0.01_0.8_16_500_500", "vgg19_0.01_0.8_16_500_500", "vgg16_0.01_0.9_16_500_500", "vgg19_0.01_0.9_16_500_500",
        "vgg16_0.001_0.7_16_500_500", "vgg19_0.001_0.7_16_500_500", "vgg16_0.001_0.8_16_500_500", "vgg19_0.001_0.8_16_500_500",
        "vgg16_0.001_0.9_16_500_500", "vgg19_0.001_0.9_16_500_500", "vgg16_0.01_0.7_16_600_600", "vgg19_0.01_0.7_16_600_600", "vgg16_0.01_0.8_16_600_600", "vgg19_0.01_0.8_16_600_600",
         "vgg16_0.01_0.9_16_600_600", "vgg19_0.01_0.9_16_600_600", "vgg16_0.001_0.7_16_600_600",
         "vgg19_0.001_0.7_16_600_600", "vgg16_0.001_0.8_16_600_600"]
nskip = ["vgg19_0.001_0.8_16_600_600", "vgg16_0.001_0.9_16_600_600", "vgg19_0.001_0.9_16_600_600", "vgg16_0.01_0.7_16_700_700", "vgg19_0.01_0.7_16_700_700",
         "vgg16_0.01_0.8_16_700_700", "vgg19_0.01_0.8_16_700_700", "vgg16_0.01_0.9_16_700_700"]
for name in os.listdir("./models"):
  if name in skip or name not in nskip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  del model

/content/drive/My Drive/Colab Notebooks
vgg19_0.001_0.8_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.8


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 252s 322ms/step - loss: 8182904.5799 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.9_16_600_600
Model: vgg16, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 220s 282ms/step - loss: 8187333.8824 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_600_600
Model: vgg19, weight >= 16MB, block size: 600 x 600, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 250s 319ms/step - loss: 8182904.5774 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.7_16_700_700
Model: vgg16, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 218s 279ms/step - loss: 47758933.5141 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0011 - sparse_top_k_categorical_accuracy: 0.0053
[48007020.0, 0.0, 0.0010003200732171535, 0.005121638998389244]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_700_700
Model: vgg19, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 249s 319ms/step - loss: 20217905.7724 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0011 - sparse_top_k_categorical_accuracy: 0.0053
[20318792.0, 0.0, 0.0010003200732171535, 0.005001600366085768]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.8_16_700_700
Model: vgg16, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 218s 279ms/step - loss: 17432039.5703 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.9284e-04 - sparse_top_k_categorical_accuracy: 0.0053
[17522328.0, 0.0, 0.0010003200732171535, 0.005261683836579323]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.8_16_700_700
Model: vgg19, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 246s 315ms/step - loss: 3667454.8379 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.9284e-04 - sparse_top_k_categorical_accuracy: 0.0050
[3684590.25, 0.0, 0.0010003200732171535, 0.005001600366085768]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.9_16_700_700
Model: vgg16, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.9


781/781 [==============================] - 219s 279ms/step - loss: 3618060.4332 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.6994e-04 - sparse_top_k_categorical_accuracy: 0.0045
[3635372.75, 0.0, 0.001300416188314557, 0.005201664753258228]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.9_16_700_700
Model: vgg19, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.9


In [ ]:
import os

print(os.getcwd())
# print(len(os.listdir("./models")))
skip = ["vgg16_0.01_0.7_16_500_500", "vgg19_0.01_0.7_16_500_500", "vgg16_0.01_0.8_16_500_500", "vgg19_0.01_0.8_16_500_500", "vgg16_0.01_0.9_16_500_500", "vgg19_0.01_0.9_16_500_500",
        "vgg16_0.001_0.7_16_500_500", "vgg19_0.001_0.7_16_500_500", "vgg16_0.001_0.8_16_500_500", "vgg19_0.001_0.8_16_500_500",
        "vgg16_0.001_0.9_16_500_500", "vgg19_0.001_0.9_16_500_500", "vgg16_0.01_0.7_16_600_600", "vgg19_0.01_0.7_16_600_600", "vgg16_0.01_0.8_16_600_600", "vgg19_0.01_0.8_16_600_600",
         "vgg16_0.01_0.9_16_600_600", "vgg19_0.01_0.9_16_600_600", "vgg16_0.001_0.7_16_600_600",
         "vgg19_0.001_0.7_16_600_600", "vgg16_0.001_0.8_16_600_600",
         "vgg19_0.001_0.8_16_600_600", "vgg16_0.001_0.9_16_600_600", "vgg19_0.001_0.9_16_600_600", "vgg16_0.01_0.7_16_700_700", "vgg19_0.01_0.7_16_700_700",
         "vgg16_0.01_0.8_16_700_700", "vgg19_0.01_0.8_16_700_700", "vgg16_0.01_0.9_16_700_700"]
nskip = ["vgg19_0.01_0.9_16_700_700", "vgg16_0.001_0.7_16_700_700", "vgg19_0.001_0.7_16_700_700", "vgg16_0.001_0.8_16_700_700", "vgg19_0.001_0.8_16_700_700", "vgg16_0.001_0.9_16_700_700",
         "vgg19_0.001_0.9_16_700_700", "vgg16_0.01_0.7_16_800_800", "vgg19_0.01_0.7_16_800_800", "vgg16_0.01_0.8_16_800_800", "vgg19_0.01_0.8_16_800_800"]
for name in os.listdir("./models"):
  if name in skip or name not in nskip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  del model

/content/drive/My Drive/Colab Notebooks
vgg19_0.01_0.9_16_700_700
Model: vgg19, weight >= 16MB, block size: 700 x 700, fp threshold: 0.01, sim threshold: 0.9


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 303s 334ms/step - loss: 3587552.3955 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 7.2619e-04 - sparse_top_k_categorical_accuracy: 0.0044
[3604523.5, 0.0, 0.0008602752932347357, 0.004901568405330181]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.7_16_700_700
Model: vgg16, weight >= 16MB, block size: 700 x 700, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 219s 280ms/step - loss: 8187333.8830 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.7_16_700_700
Model: vgg19, weight >= 16MB, block size: 700 x 700, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 250s 320ms/step - loss: 8182904.5735 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_700_700
Model: vgg16, weight >= 16MB, block size: 700 x 700, fp threshold: 0.001, sim threshold: 0.8


...

781/781 [==============================] - 251s 320ms/step - loss: 8182904.5799 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.7_16_800_800
Model: vgg16, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 218s 278ms/step - loss: 9811966.0192 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0012 - sparse_top_k_categorical_accuracy: 0.0048
[9865377.0, 0.0, 0.0010003200732171535, 0.004861555527895689]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_800_800
Model: vgg19, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 249s 318ms/step - loss: 4504759.5496 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0012 - sparse_top_k_categorical_accuracy: 0.0049
[4527937.0, 0.0, 0.0010003200732171535, 0.005301696714013815]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.8_16_800_800
Model: vgg16, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 218s 279ms/step - loss: 13717484.8734 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 4.8957e-04 - sparse_top_k_categorical_accuracy: 0.0047
[13790731.0, 0.0, 0.0006001920555718243, 0.00506161991506815]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.8_16_800_800
Model: vgg19, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 250s 319ms/step - loss: 3867319.4044 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.8988e-04 - sparse_top_k_categorical_accuracy: 0.0052
[3885454.5, 0.0, 0.0010203265119343996, 0.00504161324352026]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.9_16_800_800
Model: vgg16, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.9


In [ ]:
import os
import gc

print(os.getcwd())
# print(len(os.listdir("./models")))
skip = ["vgg16_0.01_0.7_16_500_500", "vgg19_0.01_0.7_16_500_500", "vgg16_0.01_0.8_16_500_500", "vgg19_0.01_0.8_16_500_500", "vgg16_0.01_0.9_16_500_500", "vgg19_0.01_0.9_16_500_500",
        "vgg16_0.001_0.7_16_500_500", "vgg19_0.001_0.7_16_500_500", "vgg16_0.001_0.8_16_500_500", "vgg19_0.001_0.8_16_500_500",
        "vgg16_0.001_0.9_16_500_500", "vgg19_0.001_0.9_16_500_500", "vgg16_0.01_0.7_16_600_600", "vgg19_0.01_0.7_16_600_600", "vgg16_0.01_0.8_16_600_600", "vgg19_0.01_0.8_16_600_600",
         "vgg16_0.01_0.9_16_600_600", "vgg19_0.01_0.9_16_600_600", "vgg16_0.001_0.7_16_600_600",
         "vgg19_0.001_0.7_16_600_600", "vgg16_0.001_0.8_16_600_600",
         "vgg19_0.001_0.8_16_600_600", "vgg16_0.001_0.9_16_600_600", "vgg19_0.001_0.9_16_600_600", "vgg16_0.01_0.7_16_700_700", "vgg19_0.01_0.7_16_700_700",
         "vgg16_0.01_0.8_16_700_700", "vgg19_0.01_0.8_16_700_700", "vgg16_0.01_0.9_16_700_700", 
         "vgg19_0.01_0.9_16_700_700", "vgg16_0.001_0.7_16_700_700", "vgg19_0.001_0.7_16_700_700", "vgg16_0.001_0.8_16_700_700", "vgg19_0.001_0.8_16_700_700", "vgg16_0.001_0.9_16_700_700",
         "vgg19_0.001_0.9_16_700_700", "vgg16_0.01_0.7_16_800_800", "vgg19_0.01_0.7_16_800_800", "vgg16_0.01_0.8_16_800_800", "vgg19_0.01_0.8_16_800_800"]

nskip = ["vgg16_0.01_0.9_16_800_800", "vgg19_0.01_0.9_16_800_800", "vgg16_0.001_0.7_16_800_800", "vgg19_0.001_0.7_16_800_800", "vgg16_0.001_0.8_16_800_800",
         "vgg19_0.001_0.8_16_800_800", "vgg16_0.001_0.9_16_800_800", "vgg19_0.001_0.9_16_800_800", "vgg16_0.01_0.7_16_900_900", "vgg19_0.01_0.7_16_900_900",
         "vgg16_0.01_0.8_16_900_900", "vgg19_0.01_0.8_16_900_900", "vgg16_0.01_0.9_16_900_900", "vgg19_0.01_0.9_16_900_900", "vgg16_0.001_0.7_16_900_900",
         "vgg19_0.001_0.7_16_900_900", "vgg16_0.001_0.8_16_900_900", "vgg19_0.001_0.8_16_900_900", "vgg16_0.001_0.9_16_900_900"
         "vgg19_0.001_0.9_16_900_900"]

for name in os.listdir("./models"):
  if name in skip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  tf.keras.backend.clear_session()
  del model
  gc.collect()

/content/drive/My Drive/Colab Notebooks
vgg16_0.01_0.9_16_800_800
Model: vgg16, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.9


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 266s 286ms/step - loss: 8947293.3747 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.9707e-04 - sparse_top_k_categorical_accuracy: 0.0048
[9001964.0, 0.0, 0.0008602752932347357, 0.004841549322009087]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.9_16_800_800
Model: vgg19, weight >= 16MB, block size: 800 x 800, fp threshold: 0.01, sim threshold: 0.9


781/781 [==============================] - 248s 317ms/step - loss: 3860110.5934 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0013 - sparse_top_k_categorical_accuracy: 0.0050
[3879085.5, 0.0, 0.001100352150388062, 0.005241677165031433]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.7_16_800_800
Model: vgg16, weight >= 16MB, block size: 800 x 800, fp threshold: 0.001, sim threshold: 0.7


781/781 [==============================] - 217s 277ms/step - loss: 6157544.8152 - accuracy: 0.0011 - sparse_categorical_accuracy: 0.5041 - sparse_top_k_categorical_accuracy: 0.6810
[6179283.0, 0.0010403329506516457, 0.5006002187728882, 0.6768165826797485]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.7_16_800_800
Model: vgg19, weight >= 16MB, block size: 800 x 800, fp threshold: 0.001, sim threshold: 0.7


...

781/781 [==============================] - 252s 322ms/step - loss: 4050521.6263 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0161 - sparse_top_k_categorical_accuracy: 0.0613
[4070996.0, 0.0, 0.015304897911846638, 0.05921895056962967]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_900_900
Model: vgg16, weight >= 16MB, block size: 900 x 900, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 219s 280ms/step - loss: 7243585.1675 - accuracy: 0.0010 - sparse_categorical_accuracy: 0.5842 - sparse_top_k_categorical_accuracy: 0.7765
[7271997.5, 0.0010003200732171535, 0.5828664898872375, 0.7733874917030334]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.8_16_900_900
Model: vgg19, weight >= 16MB, block size: 900 x 900, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 252s 322ms/step - loss: 6820008.1004 - accuracy: 7.7744e-04 - sparse_categorical_accuracy: 0.5821 - sparse_top_k_categorical_accuracy: 0.7723
[6850313.5, 0.0008402689127251506, 0.5776248574256897, 0.7710267305374146]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.9_16_900_900
Model: vgg16, weight >= 16MB, block size: 900 x 900, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 217s 278ms/step - loss: 7490951.2858 - accuracy: 0.0010 - sparse_categorical_accuracy: 0.6386 - sparse_top_k_categorical_accuracy: 0.8547
[7515201.5, 0.0009803137509152293, 0.637704074382782, 0.8524327874183655]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_900_900
Model: vgg19, weight >= 16MB, block size: 900 x 900, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 251s 321ms/step - loss: 6840468.6368 - accuracy: 7.7127e-04 - sparse_categorical_accuracy: 0.5927 - sparse_top_k_categorical_accuracy: 0.8103
[6870296.5, 0.0008202624740079045, 0.5878881216049194, 0.8086387515068054]
-----------------------------------------------------------------------------------------------------------------------


In [ ]:
import os

print(os.getcwd())
# print(len(os.listdir("./models")))

nskip = ["vgg16_0.01_0.7_16_400_400", "vgg19_0.01_0.7_16_400_400", "vgg16_0.01_0.8_16_400_400", "vgg19_0.01_0.8_16_400_400", "vgg16_0.01_0.9_16_400_400", "vgg19_0.01_0.9_16_400_400",
        "vgg16_0.001_0.7_16_400_400", "vgg19_0.001_0.7_16_400_400", "vgg16_0.001_0.8_16_400_400", "vgg19_0.001_0.8_16_400_400", "vgg16_0.001_0.9_16_400_400", "vgg19_0.001_0.9_16_400_400",
         ]

for name in os.listdir("./models"):
  if name not in nskip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  del model

/content/drive/My Drive/Colab Notebooks
vgg16_0.01_0.7_16_400_400
Model: vgg16, weight >= 16MB, block size: 400 x 400, fp threshold: 0.01, sim threshold: 0.7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 328s 374ms/step - loss: 34861046.1202 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0012 - sparse_top_k_categorical_accuracy: 0.0057
[35068232.0, 0.0, 0.0010003200732171535, 0.005081626120954752]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_400_400
Model: vgg19, weight >= 16MB, block size: 400 x 400, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 221s 282ms/step - loss: 15242112.8197 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0012 - sparse_top_k_categorical_accuracy: 0.0044
[15317199.0, 0.0, 0.0010003200732171535, 0.004001280292868614]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.8_16_400_400
Model: vgg16, weight >= 16MB, block size: 400 x 400, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 218s 278ms/step - loss: 18303713.6535 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0015 - sparse_top_k_categorical_accuracy: 0.0049
[18408052.0, 0.0, 0.001240396872162819, 0.004761523567140102]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.8_16_400_400
Model: vgg19, weight >= 16MB, block size: 400 x 400, fp threshold: 0.01, sim threshold: 0.8


...

781/781 [==============================] - 229s 292ms/step - loss: 6720628.9079 - accuracy: 8.9826e-04 - sparse_categorical_accuracy: 0.6271 - sparse_top_k_categorical_accuracy: 0.8462
[6744757.0, 0.0009202944929711521, 0.623919665813446, 0.8442901968955994]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_400_400
Model: vgg16, weight >= 16MB, block size: 400 x 400, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 229s 292ms/step - loss: 7002811.2820 - accuracy: 9.6416e-04 - sparse_categorical_accuracy: 0.6477 - sparse_top_k_categorical_accuracy: 0.8606
[7023104.5, 0.0009803137509152293, 0.646666944026947, 0.8584346771240234]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.8_16_400_400
Model: vgg19, weight >= 16MB, block size: 400 x 400, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 231s 295ms/step - loss: 7499111.3708 - accuracy: 9.7409e-04 - sparse_categorical_accuracy: 0.6565 - sparse_top_k_categorical_accuracy: 0.8654
[7523213.5, 0.0009603073121979833, 0.6533090472221375, 0.8644165992736816]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.9_16_400_400
Model: vgg16, weight >= 16MB, block size: 400 x 400, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 226s 289ms/step - loss: 8036631.1797 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6535 - sparse_top_k_categorical_accuracy: 0.8647
[8060125.5, 0.0009202944929711521, 0.6520086526870728, 0.8635963797569275]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_400_400
Model: vgg19, weight >= 16MB, block size: 400 x 400, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 231s 295ms/step - loss: 7915854.3843 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6593 - sparse_top_k_categorical_accuracy: 0.8670
[7940327.0, 0.0009803137509152293, 0.6560099124908447, 0.8658970594406128]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.7_16_300_300
Model: vgg16, weight >= 16MB, block size: 300 x 300, fp threshold: 0.01, sim threshold: 0.7


In [ ]:
import os
import gc

print(os.getcwd())
# print(len(os.listdir("./models")))

skip = ["vgg16_0.01_0.7_16_400_400", "vgg19_0.01_0.7_16_400_400", "vgg16_0.01_0.8_16_400_400", "vgg19_0.01_0.8_16_400_400", "vgg16_0.01_0.9_16_400_400", "vgg19_0.01_0.9_16_400_400",
        "vgg16_0.001_0.7_16_400_400", "vgg19_0.001_0.7_16_400_400", "vgg16_0.001_0.8_16_400_400", "vgg19_0.001_0.8_16_400_400", "vgg16_0.001_0.9_16_400_400", "vgg19_0.001_0.9_16_400_400",
         ]

for name in os.listdir("./models"):
  if name in skip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  tf.keras.backend.clear_session()
  del model
  gc.collect()

/content/drive/My Drive/Colab Notebooks
vgg16_0.01_0.7_16_300_300
Model: vgg16, weight >= 16MB, block size: 300 x 300, fp threshold: 0.01, sim threshold: 0.7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 263s 291ms/step - loss: 23634212.9258 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 7.9913e-04 - sparse_top_k_categorical_accuracy: 0.0045
[23751176.0, 0.0, 0.0008802816737443209, 0.004941581282764673]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_300_300
Model: vgg19, weight >= 16MB, block size: 300 x 300, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 229s 293ms/step - loss: 9498712.8760 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 0.0010 - sparse_top_k_categorical_accuracy: 0.0047
[9543161.0, 0.0, 0.0010003200732171535, 0.005201664753258228]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.8_16_300_300
Model: vgg16, weight >= 16MB, block size: 300 x 300, fp threshold: 0.01, sim threshold: 0.8


781/781 [==============================] - 233s 298ms/step - loss: 65483107.5652 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 8.0954e-04 - sparse_top_k_categorical_accuracy: 0.2049
[65910560.0, 0.0, 0.0009403008734807372, 0.20582586526870728]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.8_16_300_300
Model: vgg19, weight >= 16MB, block size: 300 x 300, fp threshold: 0.01, sim threshold: 0.8


...

781/781 [==============================] - 235s 300ms/step - loss: 7806459.0396 - accuracy: 0.0010 - sparse_categorical_accuracy: 0.6346 - sparse_top_k_categorical_accuracy: 0.8489
[7836174.0, 0.0010203265119343996, 0.6293213963508606, 0.8474311828613281]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_300_300
Model: vgg16, weight >= 16MB, block size: 300 x 300, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 233s 297ms/step - loss: 8187333.8772 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.8_16_300_300
Model: vgg19, weight >= 16MB, block size: 300 x 300, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 235s 301ms/step - loss: 8155233.5377 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6656 - sparse_top_k_categorical_accuracy: 0.8703
[8180028.0, 0.0009803137509152293, 0.6621719002723694, 0.8690580725669861]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.9_16_300_300
Model: vgg16, weight >= 16MB, block size: 300 x 300, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 233s 298ms/step - loss: 8187333.8772 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.5, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_300_300
Model: vgg19, weight >= 16MB, block size: 300 x 300, fp threshold: 0.001, sim threshold: 0.9


620/781 [======================>.......] - ETA: 48s - loss: 8174731.4113 - accuracy: 9.6217e-04 - sparse_categorical_accuracy: 0.6656 - sparse_top_k_categorical_accuracy: 0.8703

In [3]:
import os
import gc

print(os.getcwd())
# print(len(os.listdir("./models")))

skip = ["vgg16_0.01_0.7_16_400_400", "vgg19_0.01_0.7_16_400_400", "vgg16_0.01_0.8_16_400_400", "vgg19_0.01_0.8_16_400_400", "vgg16_0.01_0.9_16_400_400", "vgg19_0.01_0.9_16_400_400",
        "vgg16_0.001_0.7_16_400_400", "vgg19_0.001_0.7_16_400_400", "vgg16_0.001_0.8_16_400_400", "vgg19_0.001_0.8_16_400_400", "vgg16_0.001_0.9_16_400_400", "vgg19_0.001_0.9_16_400_400",
         "vgg16_0.01_0.7_16_300_300", "vgg19_0.01_0.7_16_300_300", "vgg16_0.01_0.8_16_300_300", "vgg19_0.01_0.8_16_300_300", "vgg16_0.01_0.9_16_300_300", "vgg19_0.01_0.9_16_300_300",
        "vgg16_0.001_0.7_16_300_300", "vgg19_0.001_0.7_16_300_300", "vgg16_0.001_0.8_16_300_300", "vgg19_0.001_0.8_16_300_300", "vgg16_0.001_0.9_16_300_300"]

for name in os.listdir("./models"):
  if name in skip:
    continue
  print(name)
  data = name.split("_")
  block_size = (int(data[-2]), int(data[-1]),)
  fp_threshold = float(data[1])
  sim_threshold = float(data[2])
  weight_lower_bound = int(data[3])
  model_name = data[0]

  print(f"Model: {model_name}, weight >= {weight_lower_bound}MB, block size: {block_size[0]} x {block_size[1]}, fp threshold: {fp_threshold}, sim threshold: {sim_threshold}")

  model = tf.keras.models.load_model(f"./models/{name}")
  model.compile('sgd', 'categorical_crossentropy', ['accuracy', 'sparse_categorical_accuracy','sparse_top_k_categorical_accuracy'])


  score = model.evaluate_generator(imagenet_generator(validation_dataset,batch_size=Nbatch), 
                                    steps= Nvalidation // Nbatch, 
                                    verbose=1)
  print(score)
  print("-----------------------------------------------------------------------------------------------------------------------")

  tf.keras.backend.clear_session()
  del model
  gc.collect()

/content/drive/My Drive/Colab Notebooks
vgg19_0.001_0.9_16_300_300
Model: vgg19, weight >= 16MB, block size: 300 x 300, fp threshold: 0.001, sim threshold: 0.9


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


781/781 [==============================] - 336s 377ms/step - loss: 8182904.5780 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.7_16_200_200
Model: vgg16, weight >= 16MB, block size: 200 x 200, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 224s 286ms/step - loss: 182811968.6138 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.0682e-04 - sparse_top_k_categorical_accuracy: 0.0045
[183747776.0, 0.0, 0.0010003200732171535, 0.005001600366085768]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.01_0.7_16_200_200
Model: vgg19, weight >= 16MB, block size: 200 x 200, fp threshold: 0.01, sim threshold: 0.7


781/781 [==============================] - 264s 337ms/step - loss: 41121274.2609 - accuracy: 0.0000e+00 - sparse_categorical_accuracy: 9.1435e-04 - sparse_top_k_categorical_accuracy: 0.0046
[41301096.0, 0.0, 0.0009803137509152293, 0.005121638998389244]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.01_0.8_16_200_200
Model: vgg16, weight >= 16MB, block size: 200 x 200, fp threshold: 0.01, sim threshold: 0.8


...

781/781 [==============================] - 265s 339ms/step - loss: 8178207.8453 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6648 - sparse_top_k_categorical_accuracy: 0.8700
[8202792.5, 0.0009803137509152293, 0.6617918014526367, 0.8691381216049194]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.8_16_200_200
Model: vgg16, weight >= 16MB, block size: 200 x 200, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 224s 287ms/step - loss: 8182436.8855 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6594 - sparse_top_k_categorical_accuracy: 0.8685
[8208980.0, 0.0009202944929711521, 0.6570102572441101, 0.8666373491287231]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.8_16_200_200
Model: vgg19, weight >= 16MB, block size: 200 x 200, fp threshold: 0.001, sim threshold: 0.8


781/781 [==============================] - 264s 338ms/step - loss: 8179110.7052 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6648 - sparse_top_k_categorical_accuracy: 0.8700
[8203704.5, 0.0009803137509152293, 0.6617717742919922, 0.8691180944442749]
-----------------------------------------------------------------------------------------------------------------------
vgg16_0.001_0.9_16_200_200
Model: vgg16, weight >= 16MB, block size: 200 x 200, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 220s 282ms/step - loss: 8187333.8632 - accuracy: 9.1185e-04 - sparse_categorical_accuracy: 0.6598 - sparse_top_k_categorical_accuracy: 0.8686
[8213870.0, 0.0009202944929711521, 0.6572903394699097, 0.8667173385620117]
-----------------------------------------------------------------------------------------------------------------------
vgg19_0.001_0.9_16_200_200
Model: vgg19, weight >= 16MB, block size: 200 x 200, fp threshold: 0.001, sim threshold: 0.9


781/781 [==============================] - 263s 336ms/step - loss: 8182904.5780 - accuracy: 9.7875e-04 - sparse_categorical_accuracy: 0.6650 - sparse_top_k_categorical_accuracy: 0.8701
[8207550.0, 0.0009803137509152293, 0.6619318127632141, 0.8690981268882751]
-----------------------------------------------------------------------------------------------------------------------
